### 성능지표 - 분류
- 정확도 (Accuracy) : 전체 정답에서 맞춘 정답의 수
- 정밀도 (Precision) : 모델이 True라고 한 것 중에 정답이 True인 비율
- 재현율 (Recall) : 정답이 True라고 한 것 중에 모델이 True인 비율
- F1-Score : 정밀도와 재현율 2가지를 조합한 점수
- 오차행렬 (Confusion Matrix) : 정답과 예측값의 관계 나타내는 표

[1] 모듈 로딩 및 데이터 준비

In [95]:
# 모듈 로딩
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score              # sklearn에서 제공하는 모델 성능평가 관련 모듈

import pandas as pd
import numpy as np

In [96]:
# 데이터 로딩
digitBunch = load_digits(as_frame=True)

[2] 피쳐와 레이블 추출

In [97]:
# 0 ~ 9까지 이미지 데이터 값
imgDF = digitBunch['data']

# 0 ~ 9 라벨
labelSR = digitBunch['target']
labelSR.unique()

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [98]:
# => 2진 분류를 위해서 7과 나머지 라벨 변경
labelSR = (labelSR == 7)
labelSR.replace({True : 1, False : 0}, inplace = True)

In [99]:
labelSR.value_counts()

target
0    1618
1     179
Name: count, dtype: int64

[3] 학습용 데이터셋 준비

In [100]:
X_train, X_test, y_train, y_test = train_test_split(imgDF, labelSR,
                                                    stratify=labelSR,
                                                    random_state=1)

In [101]:
print(y_train.value_counts() / y_train.shape[0])
print(y_test.value_counts() / y_test.shape[0])

target
0    0.90052
1    0.09948
Name: count, dtype: float64
target
0    0.9
1    0.1
Name: count, dtype: float64


[4] 클래스 정의

In [102]:
# 사용자 정의 클래스 DummyClass : 아무것도 하지 않는 클래스 단순 테스트용
from sklearn.base import BaseEstimator

class MyFakeClassifier(BaseEstimator):
    def fit(self, X, y):
        pass

    # 입력값으로 들어오는 X 데이터 세트의 크기만큼 모두 0값으로 만들어서 반환
    def predict(self, X):
        return np.zeros( (len(X), 1), dtype = bool)

[5] 예측 및 성능 평가

In [103]:
# 불균형한 레이블 데이터 분포도 확인
print('레이블 테스트 세트 크기 :', y_test.shape)
print('테스트 세트 레이블 0과 1의 분포도')
print(pd.Series(y_test).value_counts())

# Dummy Classifier로 학습/예측/정확도 평가
fakeclf = MyFakeClassifier()
fakeclf.fit(X_train, y_train)

fakepred = fakeclf.predict(X_train)
print(f"모든 예측을 0으로 하여도 정확도는:{accuracy_score(y_train, fakepred):.3f}")

레이블 테스트 세트 크기 : (450,)
테스트 세트 레이블 0과 1의 분포도
target
0    405
1     45
Name: count, dtype: int64
모든 예측을 0으로 하여도 정확도는:0.901


[6] 평가 지표

[6-1] 정확도 => 불균형 데이터 경우 신뢰 불가!!!

In [104]:
accuracy_score(y_train, fakepred)

0.9005196733481812

[6-2] 오차행렬

In [105]:
from sklearn.metrics import confusion_matrix, classification_report

In [106]:
# 정답과 예측값 전달
tn, fp, fn, tp = confusion_matrix(y_train, fakepred).reshape(-1)

print(tn, fp, fn, tp)

1213 0 134 0


In [107]:
confusion_matrix(y_train, fakepred)

array([[1213,    0],
       [ 134,    0]])

- 정밀도 (Precision) : 모델이 True라고 한 것 중에 정답이 True인 비율

In [110]:
from sklearn.metrics import precision_score
precision_score(y_train, fakepred, zero_division=0)

0.0

- 재현율 (Recall) : 정답이 True라고 한 것 중에 모델이 True인 비율

In [111]:
from sklearn.metrics import recall_score
recall_score(y_train, fakepred, zero_division=0)

0.0

In [113]:
y_test = [0,0,0,1,1,  1,0,0,0,1,  1,1,1,0,0, 0,0,0,0,0]
y_pre =  [1,1,1,1,1,  1,0,1,0,1,  1,1,1,0,0, 0,0,0,0,0]

In [114]:
# 정답과 예측값 전달
tn, fp, fn, tp = confusion_matrix(y_test, y_pre).reshape(-1)

print(tn, fp, fn, tp)

9 4 0 7


In [115]:
confusion_matrix(y_test, y_pre)

array([[9, 4],
       [0, 7]])

In [116]:
from sklearn.metrics import precision_score
precision_score(y_test, y_pre, zero_division=0)

0.6363636363636364

In [118]:
from sklearn.metrics import recall_score, f1_score
recall_score(y_test, y_pre, zero_division=0)

1.0

In [119]:
f1_score(y_test, y_pre, zero_division=0)

0.7777777777777778